<h1> Training flight delay model in BigQuery ML </h1>

Run this notebook in Vertex Workbench. In this notebook, we will use BigQuery ML to train the same model that we did in Spark ML.

Note how much easier this is ... (and also much more scaleable)

In [3]:
%load_ext bigquery_magics

## Verify dataset

Let's make sure that we have the traindays and flights data in BigQuery. If you don't, please follow steps in the README.md in this directory.

In [7]:
%%bigquery
SELECT * FROM dsongcp.flights_tzcorr
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,FL_DATE,UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_SEQ_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,...,ARR_DELAY,CANCELLED,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET
0,2015-12-25,EV,1013503,ABE,1039705,ATL,2015-12-25 11:30:00+00:00,2015-12-25 11:33:00+00:00,3.0,23.0,...,13.0,False,False,692.0,40.652222,-75.440556,-18000.0,33.636667,-84.427778,-18000.0
1,2015-05-01,OO,1013503,ABE,1039705,ATL,2015-05-01 20:00:00+00:00,2015-05-01 20:39:00+00:00,39.0,9.0,...,18.0,False,False,692.0,40.652222,-75.440556,-14400.0,33.636667,-84.427778,-14400.0
2,2015-04-07,EV,1013503,ABE,1039705,ATL,2015-04-07 16:00:00+00:00,2015-04-07 15:54:00+00:00,-6.0,14.0,...,-9.0,False,False,692.0,40.652222,-75.440556,-14400.0,33.636667,-84.427778,-14400.0
3,2015-02-19,EV,1013503,ABE,1039705,ATL,2015-02-19 21:00:00+00:00,2015-02-19 20:53:00+00:00,-7.0,12.0,...,-28.0,False,False,692.0,40.652222,-75.440556,-18000.0,33.636667,-84.427778,-18000.0
4,2015-10-28,EV,1013503,ABE,1039705,ATL,2015-10-28 16:00:00+00:00,2015-10-28 15:55:00+00:00,-5.0,12.0,...,3.0,False,False,692.0,40.652222,-75.440556,-14400.0,33.636667,-84.427778,-14400.0


In [8]:
%%bigquery
SELECT * FROM dsongcp.trainday
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,FL_DATE,is_train_day
0,2015-01-01,True
1,2015-01-02,False
2,2015-01-03,False
3,2015-01-04,True
4,2015-01-05,True


## Logistic regression

Let's use SQL to craft the dataset just the way we want it.

In [9]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,ontime,dep_delay,taxi_out,distance,is_eval_day
0,ontime,0.0,22.0,692.0,False
1,ontime,-1.0,18.0,692.0,False
2,ontime,-4.0,31.0,692.0,True
3,late,86.0,8.0,692.0,False
4,late,9.0,28.0,692.0,False


Train the model:
* ontime column in the label
* the model is a logistic regression
* use the is_eval_day column to split the data
* remaining columns are features

This will take about 10 minutes.

In [10]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Query is running:   0%|          |

""


In [11]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL dsongcp.arr_delay_lm)

Query is running:   0%|          |

Downloading:   0%|          |

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,10,0.169787,0.167228,25.6,13190
1,0,9,0.171051,0.168214,25.6,12630
2,0,8,0.173282,0.170448,51.2,12114
3,0,7,0.190024,0.186486,25.6,12459
4,0,6,0.219348,0.214750,12.8,12736
5,0,5,0.261105,0.255398,6.4,12166
6,0,4,0.321921,0.316412,3.2,12387
7,0,3,0.415229,0.411192,1.6,12860
8,0,2,0.521448,0.519101,0.8,12034
9,0,1,0.606935,0.605817,0.4,12390


In [12]:
%%bigquery
SELECT * FROM ML.WEIGHTS(MODEL dsongcp.arr_delay_lm)

Query is running:   0%|          |

Downloading:   0%|          |

,processed_input,weight,category_weights
0,dep_delay,-0.132984,[]
1,taxi_out,-0.121712,[]
2,distance,0.000223,[]
3,__INTERCEPT__,4.762562,[]


In [13]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                        (
SELECT 12.0 AS dep_delay, 14.0 AS taxi_out, 1231 AS distance
                        ))

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_ontime,predicted_ontime_probs,dep_delay,taxi_out,distance
0,ontime,"[{'label': 'ontime', 'prob': 0.850354827461451...",12.0,14.0,1231


In [14]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_lm,
                 (
                     
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
    is_train_day = 'False'
                     
                 ),
                 STRUCT(0.7 AS threshold))

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.964337,0.956538,0.935177,0.960422,0.167228,0.956261


## Do same metrics as in Spark code

We are using ML.PREDICT and computing the necessary stats

In [16]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Query is running:   0%|          |

Downloading:   0%|          |

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.83637,3907180,0.964337,851562,0.213086


## Add airport info

Add airport information to model (note two additional columns: origin and dest). This seemingly simple change adds two categorical variables that, when one-hot-encoded, adds 600+ new columns to the model. BigQuery ML doesn't skip a beat ...

This query will take ~10 minutes

In [17]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False

Query is running:   0%|          |

""


In [18]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.967152,0.957709,0.938481,0.962407,0.165552,0.960843


In [19]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Query is running:   0%|          |

Downloading:   0%|          |

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.849538,3900578,0.967152,851562,0.209835


Note that the addition of the airports information has improved both the AUC and the RMSE

Copyright 2019-2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.